## Imports

In [12]:
import pyxdf 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lmfit.models import Model
from scipy import stats
import pywt
import math

## Function Definitions

Predicted pupil dilation, $d(Y)$, caused by luminance $Y$, is computed with the following equation: $𝑑(𝑌) = 𝑎 · 𝑒^{−𝑏·𝑌} + c$

In [1]:
def pupil_func(x, a, b, c):
    return a * np.exp(-b * x) + c

In [3]:
def import_data(file):
    streams, header = pyxdf.load_xdf(file)
    dfs = {}
    for stream in streams:
        stream_name = stream['info']['name'][0]
        stream_channels = {channel['label'][0]: i for i, channel in enumerate(stream['info']['desc'][0]['channels'][0]['channel'])}
        stream_data = stream['time_series']
        data_dict = {key: np.array(stream_data)[:, index] for key, index in stream_channels.items()}
        data_dict['time'] = np.round(np.array(stream['time_stamps']), decimals=4)
        dfs[stream_name] = pd.DataFrame(data_dict).drop_duplicates(subset=['time']).reset_index(drop=True)
    return dfs

In [4]:
accom_time = pd.to_timedelta(0.5, unit='s')

In [5]:
def process_gaze_luminance_data(stream_df):
    pupil = stream_df['GazeStream'].loc[(stream_df['GazeStream']['LeftEyeIsBlinking'] == 0) & (stream_df['GazeStream']['RightEyeIsBlinking'] == 0) & (stream_df['GazeStream']['LeftPupilDiameter'] > 0) & (stream_df['GazeStream']['RightPupilDiameter'] > 0), ['time', 'MethodID', 'ModelID', 'LeftPupilDiameter', 'RightPupilDiameter']]
    pupil['time'] = pd.to_timedelta(pupil['time'], unit='s')

    lum = stream_df['LuminanceStream'].loc[:, ['time', 'MethodID', 'ModelID', 'Luminance']]
    lum['time'] = pd.to_timedelta(lum['time'], unit='s')

    # Intersection of time stamps
    pupil_lum_time_intersection = np.intersect1d(pupil['time'], lum['time'])

    # Filter pupil and luminance data by intersection
    pupil = pupil[pupil['time'].isin(pupil_lum_time_intersection)].reset_index(drop=True)
    lum = lum[lum['time'].isin(pupil_lum_time_intersection)].reset_index(drop=True)

    # Combined DataFrame for pupil and luminance
    pupil_lum = pd.DataFrame({
        'time': pd.to_timedelta(lum['time'], unit='s'),
        'luminance': lum['Luminance'],
        'pupilDiameter': 0.5 * (pupil['LeftPupilDiameter'] + pupil['RightPupilDiameter']),
        'methodID': pupil['MethodID'],
        'modelID': pupil['ModelID']
    }).resample('0.01s', on='time').mean()

    pupil_lum['time'] = pupil_lum.index

    return pupil_lum

In [6]:
def process_calibration_data(pupil_lum_df, stream_df):
    calibration_events = stream_df['ExperimentStream'].loc[(stream_df['ExperimentStream']['EventType'] == 'CalibrationColorChange') | (stream_df['ExperimentStream']['SceneEvent'] == 'Calibration') | (stream_df['ExperimentStream']['SceneEvent'] == 'CalibrationComplete'), ['time','SceneEvent', 'EventType']]
    calibration_events['time'] = pd.to_timedelta(calibration_events['time'], unit='s')
    c_start_times = calibration_events[:8]['time']
    c_end_times = calibration_events[1:]['time']
    c_start_times.reset_index(drop=True, inplace=True)
    c_end_times.reset_index(drop=True, inplace=True)

    calib_data = {}
    for i in range(8):
        calib_data[i] = pupil_lum_df.loc[(pupil_lum_df['time'] >= c_start_times[i]) & (pupil_lum_df['time'] <= c_end_times[i]), ['time','luminance', 'pupilDiameter']]
        calib_data[i]['time'] -= calib_data[i]['time'].iloc[0]
        calib_data[i] = calib_data[i].loc[(calib_data[i]['time'] >= accom_time), ['luminance', 'pupilDiameter']]

    calibration_data = pd.concat(calib_data).groupby(level=0).mean().sort_values(by=['luminance']).reset_index(drop=True)
    return calibration_data

In [62]:
def process_navigation_data(pupil_lum_df, stream_df, a, b, c):
    navigation_events = stream_df['ExperimentStream'].loc[(stream_df['ExperimentStream']['SceneEvent'] == 'NavigationComplete') | (stream_df['ExperimentStream']['SceneEvent'] == 'Navigation_Trial'), ['time','SceneEvent', 'EventType', 'ModelID', 'MethodID']]
    navigation_events['time'] = pd.to_timedelta(navigation_events['time'], unit='s')
    nav_start_times = navigation_events.loc[navigation_events['SceneEvent'] == 'Navigation_Trial', 'time']
    nav_end_times = navigation_events.loc[navigation_events['SceneEvent'] == 'NavigationComplete', 'time']

    nav_start_times.reset_index(drop=True, inplace=True)
    nav_end_times.reset_index(drop=True, inplace=True)

    #Correct for occasions when Unity emitted multiple SceneLoaded events for a single trial
    if len(nav_start_times) > 8:
        nav_diff = nav_start_times.diff().dt.total_seconds()
        nav_start_times = nav_start_times.loc[(nav_diff.isnull()) | (nav_diff > 3)]

    nav_start_times.reset_index(drop=True, inplace=True)
    nav_end_times.reset_index(drop=True, inplace=True)

    nav_data = {}
    for i in range(8):
        nav_data[i] = pupil_lum_df.loc[(pupil_lum_df['time']>=nav_start_times.loc[i]) & (pupil_lum_df['time']<=nav_end_times.loc[i]), ['time', 'methodID', 'modelID', 'luminance', 'pupilDiameter']]
        nav_data[i].set_index('time', inplace=True, drop=False)

    navigation_data = pd.concat(nav_data, names=['trial'])
    navigation_data = navigation_data.groupby(level=0).resample('0.01s', on='time', ).mean()
    navigation_data['plr'] = pupil_func(navigation_data['luminance'], a, b, c)
    navigation_data['tepr'] = navigation_data['pupilDiameter'] - navigation_data['plr']
    
    return navigation_data

In [56]:
def modmax(d):
    # compute signal modulus
    m = [0.0]*len(d)
    for i in range(len(d)):
        m[i] = math.fabs(d[i])
    # if value is larger than both neighbours , and strictly larger than either , then it is a local maximum
    t = [0.0]*len(d)
    for i in range(len(d)):
        ll = m[i -1] if i >= 1 else m[i]
        oo = m[i]
        rr = m[i+1] if i < len(d)-2 else m[i]
        if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
        # compute magnitude
            t[i] = math.sqrt(d[i]**2)
        else:
            t[i] = 0.0
    return t

In [58]:
def ipa(d):
    # obtain 2-level DWT of pupil diameter signal d
    try:
        (cA2 ,cD2 ,cD1) = pywt.wavedec(d,'sym16', 'per', level=2)
    except ValueError :
        return
    # get signal duration (in seconds)
    tt = d.index[-1] - d.index[0]
    # normalize by 1/2 j , j = 2 for 2-level DWT
    cA2 [:] = [x / math.sqrt (4.0) for x in cA2]
    cD1 [:] = [x / math.sqrt (2.0) for x in cD1]
    cD2 [:] = [x / math.sqrt (4.0) for x in cD2]

    # detect modulus maxima , see Listing 2
    cD2m = modmax(cD2)
    # threshold using universal threshold λuniv = σˆp(2logn)
    # where σˆ is the standard deviation of the noise
    λuniv = np.std(cD2m) * math.sqrt (2.0* np.log2(len(cD2m )))
    cD2t = pywt. threshold (cD2m ,λuniv,mode="hard")
    # compute IPA
    ctr = 0
    for i in range(len(cD2t )):
        if math.fabs(cD2t[i]) > 0: ctr += 1
    IPA = float(ctr)/tt.total_seconds()

    return IPA


## Load Data

Load the data from the xdf file for a single participant.

In [63]:
file = './Path_Data/ID_114.xdf'
df = import_data(file)
pupil_lum_df = process_gaze_luminance_data(df)
calibration_data = process_calibration_data(pupil_lum_df, df)

In [94]:
nav_start_times = df['NavigationStream'].groupby(['MethodID', 'ModelID']).first()
nav_start_times = pd.to_timedelta(nav_start_times['time'], unit='s')
nav_start_times = nav_start_times.reset_index(drop=True)
nav_end_times = df['NavigationStream'].groupby(['MethodID', 'ModelID']).last()
nav_end_times = pd.to_timedelta(nav_end_times['time'], unit='s')
nav_end_times = nav_end_times.reset_index(drop=True)
print(nav_start_times)
print(nav_end_times)
nav_data = {}
for i in range(8):
    nav_data[i] = pupil_lum_df.loc[(pupil_lum_df['time']>=nav_start_times.loc[i]) & (pupil_lum_df['time']<=nav_end_times.loc[i]), ['time', 'methodID', 'modelID', 'luminance', 'pupilDiameter']]
    nav_data[i].set_index('time', inplace=True, drop=False)

navigation_data = pd.concat(nav_data, names=['trial'])
navigation_data.info()

0   0 days 23:40:32.816500
1   0 days 23:41:43.825900
2   0 days 23:42:56.892300
3   0 days 23:44:07.401900
4   0 days 23:46:29.833200
5   0 days 23:47:50.628800
6   0 days 23:48:47.933700
7   0 days 23:50:06.562500
Name: time, dtype: timedelta64[ns]
0   0 days 23:41:37.218200
1   0 days 23:42:50.383700
2   0 days 23:44:00.905400
3   0 days 23:45:15.613700
4   0 days 23:47:44.142200
5   0 days 23:48:41.459400
6   0 days 23:50:00.032900
7   0 days 23:50:54.735500
Name: time, dtype: timedelta64[ns]
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 50814 entries, (0, Timedelta('0 days 23:40:32.818600')) to (7, Timedelta('0 days 23:50:54.278600'))
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   time           50814 non-null  timedelta64[ns]
 1   methodID       41451 non-null  float32        
 2   modelID        41451 non-null  float32        
 3   luminance      41451 non-null  float32        
 4  

In [67]:
navigation = process_navigation_data(pupil_lum_df, df, 0.5, 0.5, 0.5).dropna()
print(navigation[0:24])

                              methodID  modelID  luminance  pupilDiameter  \
trial time                                                                  
0     0 days 23:40:31.448600       2.0      0.0   0.000000       5.061028   
      0 days 23:40:31.468600       2.0      0.0   0.000000       5.058205   
      0 days 23:40:31.498600       2.0      0.0   0.000000       5.058891   
      0 days 23:40:31.518600       2.0      0.0   0.000000       5.058914   
      0 days 23:40:31.538600       2.0      0.0   0.000000       5.064980   
      0 days 23:40:31.558600       2.0      0.0   0.000000       5.066452   
      0 days 23:40:31.588600       2.0      0.0   0.000000       5.075096   
      0 days 23:40:31.608600       2.0      0.0   0.000000       5.078613   
      0 days 23:40:31.628600       2.0      0.0   0.000000       5.080360   
      0 days 23:40:31.648600       2.0      0.0   0.000000       5.157593   
      0 days 23:40:31.668600       2.0      0.0   0.000000       5.208229   

In [ ]:
dtime = np.array(lum['time'], dtype='datetime64')
pdil = np.array(0.5 * (pupil['LeftPupilDiameter'] + pupil['RightPupilDiameter']))

data = pd.Series(pdil, index=dtime)
ipa_data = ipa(data)
print(ipa_data)

In [43]:
x_data = calibration_data['luminance']
y_data = calibration_data['pupilDiameter']

exp_mod = Model(pupil_func)
params = exp_mod.make_params(a=1, b=4, c=0)

result = exp_mod.fit(y_data, params, x=x_data)
a = result.params['a'].value
b = result.params['b'].value
c = result.params['c'].value

# plt.plot(x_data, y_data, 'o')
# plt.plot(x_data, result.init_fit, '--', label='initial fit')
# plt.plot(x_data, result.best_fit, '-', label='best fit')
# plt.legend()
# plt.show()

In [ ]:
# idx = 1

# for i in range(8):
#     nav_data[i] = nav_data[i].resample('0.5s', origin='start').mean()

# lum_nav = nav_data[idx]['Luminance']
# pupil_nav = nav_data[idx]['pupilDiameter']

# plr = pd.DataFrame(pupil_func(lum_nav, a, b, c))
# plr['time'] = plr.index
# plr.rename(columns={"Luminance": "pupilDiameter"}, inplace=True)
# plr.reset_index(drop=True, inplace=True)

# predicted_plr = pd.DataFrame(columns=['pupilDiameter'])
# for t in range(len(plr)):
#     if t==0:
#         predicted_plr.loc[t, "pupilDiameter"] = plr.loc[t,'pupilDiameter']
#     else:
#         predicted_plr.loc[t, "pupilDiameter"] = plr.loc[t-1,'pupilDiameter']
# predicted_plr['time'] = plr['time']
# predicted_plr.set_index('time', inplace=True)

# plot_dif = pupil_nav - predicted_plr['pupilDiameter']

# x_data = pupil_nav.index

# plt.plot(x_data, predicted_plr['pupilDiameter'], '--', label='predicted')
# plt.plot(x_data, pupil_nav, '--', label='actual')
# plt.plot(x_data, plot_dif, '-', label='difference')
# plt.legend()
